<a href="https://colab.research.google.com/github/mattscocchia/NHL-Player-Ratings/blob/main/Player_Ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize

In [ ]:
!pip install cubist
!pip install --upgrade scikit-learn==1.4
#!pip install requests

import requests
import numpy as np
import pandas as pd
from scipy.stats import rankdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
# 2 minutes 30ish seconds to run

seasons=["20142015","20152016","20162017","20172018","20182019","20192020","20202021","20212022","20222023","20232024","20242025"]
teams=["ANA","ARI","BOS","BUF","CGY","CAR","CHI","COL","CBJ","DAL","DET","EDM","FLA","LAK","MIN","MTL","NSH","NJD","NYI","NYR","OTT","PHI","PIT","SJS","SEA","STL","TBL","TOR","UTA","VAN","VGK","WSH","WPG"]

def get_woodmoney_data(payload):
    url = "https://api.puckiq.com/woodmoney"  # Replace with the correct base URL
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        try:
            # Force JSON parsing even if Content-Type isn't set properly
            data = response.json()
        except ValueError:
            # If Content-Type is not application/json, handle raw response text
            data = json.loads(response.text)
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Example usage
combined_df2 = pd.DataFrame()
if __name__ == "__main__":
  for season in seasons:
    for team in teams:
        payload = {
            "season": season,
            "team": team
        }
        data = get_woodmoney_data(payload)

        if data:
            # Extract the results
            results = data.get("results", [])

            # Create a DataFrame from the results
            df = pd.DataFrame(results)

            # Print the DataFrame
            combined_df2 = pd.concat([combined_df2, df], ignore_index=False)

# 2. Concatenate the datasets and clean the data
woodmoney = combined_df2.copy()
woodmoney.rename(columns={'player_id': 'playerId'}, inplace=True)
woodmoney['season'] = woodmoney['season'].astype(str).str[:4]
woodmoney = woodmoney.groupby(['playerId', 'season', 'name', 'woodmoneytier']).sum().reset_index()

pivoted_woodmoney = woodmoney.pivot(index=['playerId', 'season', 'name'], columns='woodmoneytier', values=['ctoipct','evtoi'])

pivoted_woodmoney.columns = [
    f"ctoipct_{tier}" if col == "ctoipct" else tier
    for col, tier in pivoted_woodmoney.columns
]
pivoted_woodmoney.reset_index(inplace=True)

pivoted_woodmoney['ctoipct_All'] = 100
pivoted_woodmoney['ctoipct_Elite'] = pivoted_woodmoney['Elite']/pivoted_woodmoney['All']
pivoted_woodmoney['ctoipct_Middle'] = pivoted_woodmoney['Middle']/pivoted_woodmoney['All']
pivoted_woodmoney['ctoipct_Gritensity'] = pivoted_woodmoney['Gritensity']/pivoted_woodmoney['All']


# Define weight mapping for each woodmoneytier
tier_weights = {'Elite': 0.9, 'Middle': 0.5, 'Gritensity': 0.2, 'All': 1}

# 1. Apply weights to each column directly
pivoted_woodmoney['Elite_weighted'] = pivoted_woodmoney['Elite'] * tier_weights['Elite']
pivoted_woodmoney['Middle_weighted'] = pivoted_woodmoney['Middle'] * tier_weights['Middle']
pivoted_woodmoney['Gritensity_weighted'] = pivoted_woodmoney['Gritensity'] * tier_weights['Gritensity']
# For 'All', keep it unweighted (raw TOI)
pivoted_woodmoney['All_weighted'] = pivoted_woodmoney['All']

# 2. Calculate total weighted TOI across tiers (excluding 'All')
pivoted_woodmoney['total_weighted_toi'] = (
    pivoted_woodmoney['Elite_weighted'] +
    pivoted_woodmoney['Middle_weighted'] +
    pivoted_woodmoney['Gritensity_weighted']
)

# 3. Calculate the final opponent strength as a weighted average
pivoted_woodmoney['minute_strength'] = (
    pivoted_woodmoney['total_weighted_toi'] /
    1.6
)

# 4. Keep only relevant columns for the summary
opponent_strength_summary = pivoted_woodmoney[['playerId', 'name', 'season', 'minute_strength']]

opponent_strength_summary['season'] = pd.Categorical(opponent_strength_summary['season'])
opponent_strength_summary['season'] = opponent_strength_summary['season'].astype(int)

In [ ]:
# Load the data into a pandas dataframe
df = pd.read_csv('all_players.csv')
cut_year = 2016
data = df[df['season'] >= (cut_year-2)].copy()
data['season'] = pd.Categorical(data['season'])

data = data.dropna()

games_played_req = 30

new_data = pd.merge(data[(data['situation'] == '5on5') & (data['games_played'] > games_played_req)], opponent_strength_summary[['playerId', 'season', 'minute_strength']], on=['playerId','season'], how='left')

teams = pd.read_csv("all_teams.csv")
new_data['goalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','goalsAgainst','season']], on=['team','season'], how='left')['goalsAgainst']
new_data['goalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','goalsFor','season']], on=['team','season'], how='left')['goalsFor']
new_data['xGoalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','xGoalsAgainst','season']], on=['team','season'], how='left')['xGoalsAgainst']
new_data['xGoalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','xGoalsFor','season']], on=['team','season'], how='left')['xGoalsFor']
new_data['team_icetime'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','iceTime','season']], on=['team','season'], how='left')['iceTime']
new_data['pk_team_icetime'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','iceTime','season']], on=['team','season'], how='left')['iceTime']
new_data['pp_team_icetime'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','iceTime','season']], on=['team','season'], how='left')['iceTime']
new_data['pp_goalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','goalsAgainst','season']], on=['team','season'], how='left')['goalsAgainst_y']
new_data['pp_goalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','goalsFor','season']], on=['team','season'], how='left')['goalsFor_y']
new_data['pp_xgoalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','xGoalsAgainst','season']], on=['team','season'], how='left')['xGoalsAgainst_y']
new_data['pp_xgoalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','xGoalsFor','season']], on=['team','season'], how='left')['xGoalsFor_y']
new_data['pk_xgoalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','xGoalsAgainst','season']], on=['team','season'], how='left')['xGoalsAgainst_y']
new_data['pk_xgoalsFor'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','xGoalsFor','season']], on=['team','season'], how='left')['xGoalsFor_y']
new_data['pk_goalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','goalsAgainst','season']], on=['team','season'], how='left')['goalsAgainst_y']
new_data['pk_goalsFor'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','goalsFor','season']], on=['team','season'], how='left')['goalsFor_y']
new_data['team_games'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','games_played','season']], on=['team','season'], how='left')['games_played_y']
new_data['xGoalsp'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','xGoalsPercentage','season']], on=['team','season'], how='left')['xGoalsPercentage']

icetime = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['icetime']).reset_index()['icetime']
timeOnBench = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['timeOnBench']).reset_index()['timeOnBench']
oigf = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_F_goals']).reset_index()['OnIce_F_goals']
oiga = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_A_goals']).reset_index()['OnIce_A_goals']
oixgf = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_F_xGoals']).reset_index()['OnIce_F_xGoals']
oixga = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_A_xGoals']).reset_index()['OnIce_A_xGoals']
penaltiesT = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['penalties']).reset_index()['penalties']
penaltiesD = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['penaltiesDrawn']).reset_index()['penaltiesDrawn']
takeaways = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_takeaways']).reset_index()['I_F_takeaways']
giveaways = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_giveaways']).reset_index()['I_F_giveaways']
points = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_points']).reset_index()['I_F_points']
I_F_primaryAssists = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_primaryAssists']).reset_index()['I_F_primaryAssists']
I_F_secondaryAssists = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_secondaryAssists']).reset_index()['I_F_secondaryAssists']
I_F_goals = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_goals']).reset_index()['I_F_goals']
I_F_xGoals = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_xGoals']).reset_index()['I_F_xGoals']

new_data['pk_time'] = icetime['4on5']
new_data['pk_time_bench'] = icetime['4on5']
new_data['pp_time'] = icetime['5on4']
new_data['pk_gf'] = oigf['4on5']
new_data['pp_gf'] = oigf['5on4']
new_data['pk_ga'] = oiga['4on5']
new_data['pp_ga'] = oiga['5on4']
new_data['pk_xgf'] = oixgf['4on5']
new_data['pp_xgf'] = oixgf['5on4']
new_data['pk_xga'] = oixga['4on5']
new_data['pp_xga'] = oixga['5on4']
new_data['penalties'] = penaltiesT['all']
new_data['penaltiesDrawn'] = penaltiesD['all']
new_data['takeaways'] = takeaways['all']
new_data['giveaways'] = giveaways['all']
new_data['pp_points'] = points['5on4']
new_data['I_F_primaryAssists'] = I_F_primaryAssists['all']
new_data['I_F_secondaryAssists'] = I_F_secondaryAssists['all']
new_data['I_F_goals'] = I_F_goals['all']
new_data['I_F_xGoals'] = I_F_xGoals['all']

from scipy.stats import rankdata
# Combine the two dataframes
combined_df = new_data.copy()

# Step 1: Assign values to 'ford' based on 'position'
def determine_ford(position):
    if "D" in position:
        return "D"
    else:
        return "F"

combined_df["ford"] = combined_df["position"].apply(determine_ford)

# Step 2: Calculate gameScore_percentile by grouping
combined_df["gameScore_percentile"] = (
    combined_df.groupby(["situation", "season", "ford"])["gameScore"]
    .rank(pct=True)
)

# Step 2: Calculate gameScore_percentile by grouping
combined_df["overall_percentile"] = (
    combined_df.groupby(["situation", "season"])["gameScore"]
    .rank(pct=True)
)

combined_df['minute_strength_percentile'] = (
    combined_df.groupby(["situation", "season", "ford"])['minute_strength']
    .rank(pct=True)
)

# Split back the percentiles into the original dataframes
new_data['gameScore_percentile'] = combined_df.loc[:, 'gameScore_percentile'].values

new_data['overall_percentile'] = combined_df.loc[:, 'overall_percentile'].values

new_data['minute_strength_percentile'] = combined_df.loc[:, 'minute_strength_percentile'].values

## Data Prep

In [ ]:
# Calculate each component

### FINAL COPY

new_data['Component_1'] = 0.02*new_data['shotsBlockedByPlayer']
new_data['Component_2'] = 0.015 * (new_data['faceoffsWon'] - new_data['faceoffsLost'])
new_data['Component_3'] = 0.06 * ( - new_data['penalties'])
xga_d = 2*((((new_data['OnIce_A_xGoals'] / new_data['icetime']) -
     (new_data.groupby("season")["xGoalsAgainst"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_4'] = np.where(
    new_data['ford'] == 'F',
    1.75 * xga_d,
    2.3 * xga_d
)
ga_d = 2*((((new_data['OnIce_A_goals'] / new_data['icetime']) -
     (new_data.groupby("season")["goalsAgainst"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_5'] = np.where(
    new_data['ford'] == 'F',
    0.4375 * ga_d,
    0.575 * ga_d
)
new_data['Component_6'] = new_data['pk_time'] / 60 / 60
new_data['Component_7'] = (0.1*(( new_data['pk_ga']/new_data['pk_time'] - (new_data.groupby("season")["pk_goalsAgainst"].transform("mean")/new_data.groupby("season")["pk_team_icetime"].transform("mean")))*60*60).fillna(0) + 0.1*((new_data['pk_xga']/new_data['pk_time'] - (new_data.groupby("season")["pk_xgoalsAgainst"].transform("mean")/new_data.groupby("season")["pk_team_icetime"].transform("mean")))*60*60).fillna(0))
new_data['pk_rating'] = np.where(
    new_data['Component_6'] > 0.3,
    -2*((0.5*new_data['Component_7']))*new_data['Component_6'],
    0
)
base_rating = -2*(new_data['Component_4'] + new_data['Component_5'])-(new_data['Component_4'] - new_data['Component_5'])*0.1

# Apply condition: if base_rating < 0, multiply by (1 - minute_strength_percentile), otherwise multiply by minute_strength_percentile
new_data['5on5_drating'] = np.where(
    base_rating < 0,
    base_rating * (1 - new_data['minute_strength_percentile']),
    base_rating * new_data['minute_strength_percentile']
)


# Step 2: Calculate gameScore_percentile by grouping
new_data["5on5_drating_pct"] = (
    new_data.groupby(["season"])["5on5_drating"]
    .rank(pct=True)
)

def_rating = (
    new_data['Component_1'] +
    new_data['Component_2'] +
    new_data['Component_3'] +
    new_data['5on5_drating'] +
    new_data['pk_rating'] +
    0.01 * new_data['I_F_takeaways']
) * (new_data['games_played']/new_data['team_games'])

new_data['def_rating'] = np.where(
    new_data['ford'] == 'F',
    def_rating,
    def_rating-0.5
)

# Step 2: Calculate gameScore_percentile by grouping
new_data["def_percentile"] = (
    new_data.groupby(["season","ford"])["def_rating"]
    .rank(pct=True)
)

# Display the components and the result
components_table = new_data[['name','season', 'games_played', 'team', 'ford','minute_strength_percentile', 'Component_1', 'Component_2', 'Component_3', 'Component_4',
                             'Component_5', 'Component_6', 'Component_7','pk_rating','5on5_drating', '5on5_drating_pct', 'def_rating', 'def_percentile','I_F_takeaways']]#[new_data['season'] == 2023]

components_table

,name,season,position,off_percentile,off_rating,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,Component_10,Component_11,Component_13
0,Jaromir Jagr,2014,R,0.909297,26.303856,9.00,7.0,2.10,-0.0460,0.00,0.80,0.9338,0.36,2.828889,1.46,5.572167,2.005
1,Jaromir Jagr,2015,R,0.991091,38.970006,15.00,8.0,4.90,0.3970,-0.01,0.60,1.0632,2.98,3.424722,0.68,5.525083,1.690
2,Jaromir Jagr,2016,R,0.934611,28.152261,6.00,8.5,2.80,-0.2500,0.00,0.90,1.1164,2.22,3.476944,1.54,5.608917,2.110
3,Jaromir Jagr,2017,R,0.326966,-1.761094,0.75,1.5,0.70,-0.0700,0.00,0.00,-0.0954,-4.20,0.553889,-1.18,1.250417,0.420
4,Sergei Gonchar,2014,D,0.477324,4.261683,0.75,2.0,1.75,0.0010,0.01,0.00,0.0736,-2.76,1.529167,-0.28,3.552917,1.165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9954,Juha Jaaska,2024,L,0.085158,-4.790200,0.00,0.5,0.00,-0.0290,0.01,0.00,0.0278,-4.18,0.017500,-1.24,0.263500,0.160
9955,Arttu Hyry,2024,R,0.094891,-4.677983,0.00,0.5,0.00,-0.0075,0.06,0.05,-0.2764,-4.02,0.000000,-1.08,0.225917,0.080
9956,Maxim Tsyplakov,2024,R,0.789538,10.640344,4.50,2.0,2.10,-0.1695,0.00,0.25,0.7634,0.06,1.289444,0.18,3.032000,1.015
9957,Jere Innala,2024,L,0.049878,-5.160094,0.00,0.0,0.00,-0.0310,-0.02,0.10,-0.4744,-3.90,0.030556,-1.08,0.384750,0.120


In [ ]:

# Calculate each component
new_data['Component_1'] = np.where(
    new_data['ford'] == 'F',
    0.3 * new_data['I_F_goals'],
    0.4 * new_data['I_F_goals']
)

new_data['Component_2'] = 0.4 * new_data['I_F_primaryAssists']
new_data['Component_3'] = 0.25 * new_data['I_F_secondaryAssists']

new_data['Component_8'] = np.where(
    new_data['ford'] == 'F',
    0.25 * new_data['I_F_xGoals'],
    0.15 * new_data['I_F_xGoals']
)

# Team xGoalsFor - Individual xGoalsFor = offIceXGoalsFor
xgf_o = 2*((((new_data['OnIce_F_xGoals'] / new_data['icetime']) -
     (new_data.groupby("season")["xGoalsFor"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_4'] = np.where(
    new_data['ford'] == 'F',
    0.625 * xgf_o,
    1.7 * xgf_o
)
gf_o = 2*((((new_data['OnIce_F_goals'] / new_data['icetime']) -
     (new_data.groupby("season")["goalsFor"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_5'] = np.where(
    new_data['ford'] == 'F',
    0.625 * gf_o,
    0.425 * gf_o
)
new_data['Component_6'] = new_data['pp_time'] / 60 / 60
new_data['Component_7'] = (0.1*((new_data['pp_gf']/new_data['pp_time'] - (new_data.groupby("season")["pp_goalsFor"].transform("mean")/new_data.groupby("season")["pp_team_icetime"].transform("mean")))*60*60).fillna(0) + 0.1*(((-new_data['pp_xgf']/new_data['pp_time'] - (new_data.groupby("season")["pp_xgoalsFor"].transform("mean")/new_data.groupby("season")["pp_team_icetime"].transform("mean"))))*60*60).fillna(0))
new_data['pp_rating'] = np.where(
    new_data['Component_6'] > 0.3,
    -2*((0.5*new_data['Component_7']))*new_data['Component_6'],
    0
)
base_rating = (new_data['Component_4'] + new_data['Component_5'])-(new_data['Component_4'] - new_data['Component_5'])*0.1

# Apply condition: if base_rating < 0, multiply by (1 - minute_strength_percentile), otherwise multiply by minute_strength_percentile
new_data['5on5_orating'] = np.where(
    base_rating < 0,
    base_rating * (1 - new_data['minute_strength_percentile']),
    base_rating * new_data['minute_strength_percentile']
)


# Step 2: Calculate gameScore_percentile by grouping
new_data["5on5_orating_pct"] = (
    new_data.groupby(["season"])["5on5_orating"]
    .rank(pct=True)
)

off_rating = (
    new_data['Component_1'] +
    new_data['Component_2'] +
    new_data['Component_3'] +
    new_data['Component_8'] +
    new_data['5on5_orating'] +
    new_data['pp_rating'] +
    0.06 * (new_data['penaltiesDrawn']) -
    0.01 * new_data['giveaways']
) * (new_data['games_played']/new_data['team_games']) /1.5

new_data['off_rating'] = np.where(
    new_data['ford'] == 'F',
    off_rating-8,
    off_rating-4
)

# Step 2: Calculate gameScore_percentile by grouping
new_data["off_percentile"] = (
    new_data.groupby(["season","ford"])["off_rating"]
    .rank(pct=True)
)

# Display the components and the result
components_table = new_data[['name','season', 'games_played', 'team', 'ford','minute_strength_percentile', 'Component_1', 'Component_2', 'Component_3', 'Component_4',
                             'Component_5', 'Component_6', 'Component_7','Component_8','pp_rating','5on5_orating', '5on5_orating_pct', 'off_rating', 'off_percentile']]#[new_data['season'] == 2021]

components_table

,name,season,games_played,team,ford,minute_strength_percentile,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,def_rating,def_percentile
501,Ryan Suter,2024,45,STL,D,0.849650,3.22,0.00,-1.20,-2.473902,-2.438915,0.90,1.518056,-0.072811,1.330,0.598311,0.482968
574,Brent Burns,2024,44,CAR,D,0.930070,2.87,0.00,-0.75,-1.955093,-2.818050,2.55,1.695556,-0.007407,0.890,2.025697,0.763990
602,Corey Perry,2024,42,EDM,F,0.457090,0.77,-0.01,-1.05,1.232525,1.923680,1.05,0.000000,0.049411,0.575,1.778973,0.720195
762,Alex Ovechkin,2024,28,WSH,F,0.434701,0.35,0.00,-0.60,-0.926650,1.115786,0.75,0.011667,0.049411,0.560,0.325458,0.408759
773,Evgeni Malkin,2024,42,PIT,F,0.787313,1.82,-0.52,-1.20,-2.214975,-2.576320,1.80,0.005000,0.049411,0.810,-1.314183,0.018248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9954,Juha Jaaska,2024,6,CAR,F,0.164179,0.14,0.01,-0.15,0.487146,0.524811,0.00,0.000833,0.049411,0.105,0.022996,0.262774
9955,Arttu Hyry,2024,5,DAL,F,0.098881,0.21,0.06,-0.00,-0.034045,0.395676,0.15,0.000000,0.049411,0.065,0.009041,0.226277
9956,Maxim Tsyplakov,2024,43,NYI,F,0.789179,1.26,0.00,-1.65,-0.816284,0.052815,2.40,0.001111,0.049411,0.880,1.539118,0.676399
9957,Jere Innala,2024,11,COL,F,0.143657,0.28,-0.02,-0.15,0.810602,0.920488,0.15,0.002500,0.049411,0.135,0.067485,0.298054


In [ ]:
new_data['ovr_rating'] = new_data['off_rating'] + new_data['def_rating']

# Step 2: Calculate gameScore_percentile by grouping
new_data["ovr_ptile"] = (
    new_data.groupby(["season","ford"])["ovr_rating"]
    .rank(pct=True)
)

new_data['ovr_pg'] = new_data['ovr_rating']/new_data['games_played']

new_data['ovr_rating_pr'] = new_data['ovr_pg']*82

# Step 2: Calculate gameScore_percentile by grouping
new_data["ovr_ptile_pr"] = (
    new_data.groupby(["season","ford"])["ovr_rating_pr"]
    .rank(pct=True)
)

# Step 3: Calculate Percentiles for Summed Data
components_table = new_data[['name','season', 'ford', 'games_played', 'team', 'off_rating', 'def_rating', 'off_percentile', 'def_percentile', 'ovr_rating', 'ovr_ptile','ovr_pg', 'ovr_rating_pr', 'ovr_ptile_pr']]

components_table

,name,season,ford,games_played,team,position,minute_strength_percentile,off_rating,def_rating,off_percentile,def_percentile,ovr_rating,ovr_ptile
0,Jaromir Jagr,2014,F,77,FLA,R,0.564236,26.303856,5.074951,0.909297,0.742630,14.843578,0.896825
1,Jaromir Jagr,2015,F,79,FLA,R,0.841484,38.970006,4.490294,0.991091,0.713808,20.981767,0.984410
2,Jaromir Jagr,2016,F,81,FLA,R,0.779287,28.152261,4.077079,0.934611,0.696731,15.435157,0.907554
3,Jaromir Jagr,2017,F,22,CGY,R,0.395548,-1.761094,0.434741,0.326966,0.339326,-0.735634,0.323596
4,Sergei Gonchar,2014,D,48,MTL,D,0.339869,4.261683,1.444266,0.477324,0.444444,2.612264,0.456916
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9954,Juha Jaaska,2024,F,6,CAR,L,0.072761,-4.790200,0.010235,0.085158,0.183698,-2.391688,0.082725
9955,Arttu Hyry,2024,F,5,DAL,R,0.555970,-4.677983,0.055373,0.094891,0.259124,-2.320534,0.097324
9956,Maxim Tsyplakov,2024,F,43,NYI,R,0.841418,10.640344,2.244014,0.789538,0.794404,6.068177,0.793187
9957,Jere Innala,2024,F,11,COL,L,0.037313,-5.160094,0.017850,0.049878,0.211679,-2.574097,0.048662


In [ ]:
# Step 1: Create overall rating
new_data['ovr_rating'] = new_data['off_rating'] + new_data['def_rating']

# Step 2: Aggregate over multiple seasons
summed_data = (
    new_data[(new_data['season'] > 2021) & (new_data['season'] <= 2024)]
    .groupby(["playerId","ford","name"], as_index=False)  # Group by player and team/ford
    .agg({
        "games_played": "sum",
        "off_rating": "sum",
        "def_rating": "sum",
        "ovr_rating": "sum"
    })
)

summed_data['ovr_pg'] = summed_data['ovr_rating']/summed_data['games_played']

# Step 3: Calculate Percentiles for Summed Data
summed_data["ovr_ptile"] = summed_data.groupby("ford")["ovr_rating"].rank(pct=True)

# Step 4: Create the Final Table
components_table = summed_data[['name', 'ford', 'games_played', 'off_rating', 'def_rating', 'ovr_rating', 'ovr_ptile', 'ovr_pg']]

# Display the table
components_table


In [ ]:
# Step 1: Create overall rating
new_data['ovr_rating'] = new_data['off_rating'] + new_data['def_rating']

# Step 2: Aggregate over multiple seasons
summed_data = (
    new_data#[(new_data['season'] > 2021) & (new_data['season'] <= 2024)]
    .groupby(["team","season"], as_index=False)  # Group by player and team/ford
    .agg({
        "games_played": "sum",
        "off_rating": "sum",
        "def_rating": "sum",
        "ovr_rating": "sum",

    })
)

summed_data['ovr_pg'] = summed_data['ovr_rating']/summed_data['games_played']

# Step 3: Calculate Percentiles for Summed Data
summed_data["ovr_ptile"] = summed_data.groupby("season")["ovr_rating"].rank(pct=True)
summed_data["ovr"] = summed_data.groupby("season")["ovr_rating"].rank(ascending=False)

# Step 4: Create the Final Table
components_table = summed_data[['team','season', 'games_played', 'off_rating', 'def_rating', 'ovr_rating', 'ovr_ptile', 'ovr', 'ovr_pg']]

# Display the table
components_table
